In [ ]:
import numpy as np
import sys
import os
import pandas as pd
sys.path.append('../../')

from script.Model_run import Speech_analysis

In [ ]:
#Directory where the data are stored
data_home_dir = '/home/kvattis/Dropbox (Partners HealthCare)/Data_Raw_Biogen_SCA/'
data_home_dir_at = '/home/kvattis/Dropbox (Partners HealthCare)/Data_Raw_A-T_Remote_Study/'


In [ ]:
speech_analyzer = Speech_analysis(print_on = False)

In [ ]:
speech_analyzer.class_map[0]

In [ ]:
IDs_used = list({x for l in speech_analyzer.class_map for x in l})

In [ ]:
len(IDs_used)

# Biogen SCA study

In [ ]:
#survey_lex = pd.read_csv(data_home_dir + 'Biogen_SCA_Speech_Data/' + 'surveylex_questionIDs.csv',encoding='latin1')
survey_lex = pd.read_excel(data_home_dir + 'Biogen_SCA_Speech_Data/' + 'surveylex_questionIDs.xlsx',engine='openpyxl')
taskslist = survey_lex['question'].unique()
output = pd.DataFrame(columns = ['Date','subject_id','Neuropheno_ID','Bars','Bars_Speech','Sex','Diagnosis','DoB','Samples_no','Prob_AT_tf_grad', 'Bars_Speech_pred_t','Bars_Total_pred_t'])
                      
for tt in  taskslist: 
    if tt not in ["la_la_la","go_go_go","me_me_me", "lalala", "gogogo", "mememe","papapa"]:
        continue
    print(tt)
    pd.set_option('display.max_columns', 4)
    np.dtype(np.int16)
    task = str(tt)
    task_dir = str(tt)
    
    study_name = 'Biogen_SCA_Speech_Data'
    study_dir = data_home_dir + 'Biogen_SCA_Speech_Data/'
    #survey_lex = pd.read_csv(study_dir + 'surveylex_questionIDs.csv',encoding='latin1')
    survey_lex = pd.read_excel(study_dir + 'surveylex_questionIDs.xlsx',engine='openpyxl')

    survey_response_A = pd.read_excel(study_dir + 'surveyA-responses-summary_.xlsx',engine='openpyxl')
    survey_response_B = pd.read_excel(study_dir + 'surveyB-responses-summary_.xlsx',engine='openpyxl')
    survey_response_C = pd.read_excel(study_dir + 'surveyC-responses-summary_.xlsx',engine='openpyxl')
    survey_response_D = pd.read_excel(study_dir + 'surveyD-responses-summary_.xlsx',engine='openpyxl')
    
    survey_response = pd.concat([survey_response_A, survey_response_B, survey_response_C, survey_response_D], ignore_index=True)

    survey_submissions_A = pd.read_excel(study_dir + 'surveyA-submissions-summary_.xlsx',engine='openpyxl')
    survey_submissions_B = pd.read_excel(study_dir + 'surveyB-submissions-summary_.xlsx',engine='openpyxl')
    survey_submissions_C = pd.read_excel(study_dir + 'surveyC-submissions-summary_.xlsx',engine='openpyxl')
    survey_submissions_D = pd.read_excel(study_dir + 'surveyD-submissions-summary_.xlsx',engine='openpyxl')
    

    survey_submissions = pd.concat([survey_submissions_A, survey_submissions_B, survey_submissions_C, survey_submissions_D], ignore_index=True)

    #Patient_info = pd.read_csv(study_dir + 'Subjects_BiogenSCA_version2.csv',encoding='latin1')
    #Patient_info = pd.read_csv(data_home_dir + 'Subjects_BiogenSCA_version2.csv',encoding='latin1')
    #Patient_info_ = Patient_info.loc[:,['subject_id','neuropheno_id','diagnosis','bars_total_excl_miss_1','bars_total_excl_miss_2', 'bars_speech_1', 'bars_speech_2','sex', 'age']]
    
    _Patient_info = pd.read_csv(data_home_dir + 'Subjects/Subjects_BiogenSCA_version2.csv',encoding='latin1')
    _Patient_info_ = _Patient_info.loc[:,['subject_id','neuropheno_id','diagnosis','bars_total_excl_miss_1','bars_total_excl_miss_2', 'bars_speech_1', 'bars_speech_2','sex']]

    Patient_info = pd.read_excel(data_home_dir + 'Subjects/Subjects_BiogenSCA_with_questionnaires_2022_05_04_converted.xlsx',engine='openpyxl')
    Patient_info_ = Patient_info.loc[:,['subject_id','neuropheno_id','diagnosis','bars_total_excl_miss_1','bars_total_excl_miss_2', 'bars_speech_1', 'bars_speech_2','sex','dob']]
    
    for i in _Patient_info_['subject_id']:
        Patient_info_.loc[Patient_info_['subject_id'] == i,'neuropheno_id']  = _Patient_info_.loc[_Patient_info_['subject_id'] == i,'neuropheno_id']
    
    survey_lex_ = survey_lex.loc[:,['questionId','question']]
    survey_response_ = survey_response.loc[:,['sampleId','questionId', 'sessionId','createdDate']]
    survey_response_['createdDate'] = survey_response_['createdDate'].apply(lambda x: x[0:4] + '_' + x[5:7] + '_' + x[8:10] )
    survey_submissions_ = survey_submissions.loc[:,['sessionId','Your participant ID:']]
    survey_submissions_ = survey_submissions_.rename(columns={"Your participant ID:": "subject_id"})
    df = pd.merge(pd.merge(pd.merge(survey_response_,survey_lex_,on='questionId'), survey_submissions_, on = 'sessionId'),Patient_info_, on = 'subject_id')
    df.dropna(subset = ["sampleId"], inplace=True)

    df_task = df.loc[df['question'] == task][['sampleId','subject_id','neuropheno_id','diagnosis','createdDate','bars_total_excl_miss_1','bars_total_excl_miss_2', 'bars_speech_1', 'bars_speech_2','sex']]#, 'age']]
    df_task_numpy = df_task.to_numpy()

    
        
    list_diagnostic = ['Control', 'Ataxia', 'MSA', 'Other']

    
    #Going over all the files for the task
    list_names = []
    for entry in df_task_numpy:
        file_name = entry[0]
        ID = str(entry[1])

        if  not np.isnan(entry[2]):
            Neur_phen_id = str(int(entry[2]))
        else:
            Neur_phen_id = ID
            
        if int(Neur_phen_id) in IDs_used:
            continue
        
        diagn = entry[3]
        date = entry[4]
        Date = date[:4] + date[5:7] + date[8:]
        bars_total = (entry[5] + entry[6])/2.
        bars_speech = (entry[7] + entry[8])/2./10.
        sex = entry[9]
        #age = entry[10]
        
        path_to_file = study_dir + ID + '_' + date + '_A/' + file_name + '.wav'
        path_to_file_flag = 0
        
        if os.path.exists(path_to_file):
            path_to_file_flag = 1 
        
        if not os.path.exists(path_to_file):
            path_to_file = study_dir + ID + '_' + date + '_B/' + file_name + '.wav'
        else:
            path_to_file_flag = 1
        
        if not os.path.exists(path_to_file):
            path_to_file = study_dir + ID + '_' + date + '_C/' + file_name + '.wav'
        else:
            path_to_file_flag = 1
            
        if not os.path.exists(path_to_file):
            path_to_file = study_dir + ID + '_' + date + '_D/' + file_name + '.wav'
        else:
            path_to_file_flag = 1    
            
        if path_to_file_flag == 0: continue;
            
        #Assign label
        if diagn == 0:
            #diagn = 'Control'
            label = 0
        elif diagn == 1:
            #diagn = 'Ataxia'
            label = 1
        elif diagn == 2:
            #diagn = 'Ataxia'
            label = 1
        elif diagn == 3:
            #diagn = 'Ataxia'
            label = 1
        elif diagn == 7:
            #diagn = 'MSA'
            label = 7
        else:
            #diagn = 'Other'
            label = 5
        
        if label not in [0,1]:
            continue
        
        if label == 0:
            bars_total = 0
            bars_speech = 0 
        
        if Neur_phen_id == ID:
            #print(path_to_file)
            no_samples = speech_analyzer.load(path_to_file)
            Neur_phen_id = None
        else:
            no_samples = speech_analyzer.load(path_to_file,Neur_phen_id)
        
        if no_samples == 0: 
            continue
            
        output = output.append({'Date' : Date, 'subject_id' : ID, 'Neuropheno_ID' : Neur_phen_id, 'Bars': bars_total, 'Bars_Speech':bars_speech, 'Sex': sex, 'Diagnosis': diagn,'Samples_no' : speech_analyzer.sample_size_s, 'Prob_AT_tf_grad': speech_analyzer.classify(),
                               'Bars_Speech_pred_t': speech_analyzer.BARS_speech(), 'Bars_Total_pred_t': speech_analyzer.BARS_total()}, 
                ignore_index = True)
        
output = output.astype(float).groupby(['Date','subject_id'], as_index = False).median().sort_values(by=['subject_id'])

In [ ]:
speech_analyzer.plot_wav()

In [ ]:
speech_analyzer.play_audio_nr()

In [ ]:
speech_analyzer.plot_mel()

In [ ]:
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 200)
np.dtype(np.int16)

In [ ]:
output.head(200)

In [ ]:
for i in output['subject_id'].unique():
    output.loc[output['subject_id'] == i,'DoB'] = pd.to_datetime(Patient_info_.loc[Patient_info_['subject_id'] == i,'dob'].item())


In [ ]:
output[['subject_id','Bars', 'Bars_Speech', 'Samples_no', 'Diagnosis','Prob_AT_tf_grad', 'Bars_Speech_pred_t','Bars_Total_pred_t']].head(200)

output.to_csv('/home/kvattis/Desktop/BiogenSca_outputs_2022_04_22.csv', index = False)

In [ ]:
output_sca1 = output[output['Diagnosis'] == 1]
output_sca2 = output[output['Diagnosis'] == 2]
output_sca3 = output[output['Diagnosis'] == 3]
output_c = output[output['Diagnosis'] == 0]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 1, 1)
ax.scatter(output_c['Bars_Speech'], output_c['Prob_AT_tf_grad'], c = 'b' , marker = 'x')
ax.scatter(output_sca1['Bars_Speech'], output_sca1['Prob_AT_tf_grad'], c = 'k' , marker = 'x')
ax.scatter(output_sca2['Bars_Speech'], output_sca2['Prob_AT_tf_grad'], c = 'g' , marker = 'x')
ax.scatter(output_sca3['Bars_Speech'], output_sca3['Prob_AT_tf_grad'], c = 'r' , marker = 'x')
plt.xlim([-0.2, 4])
plt.ylim([-0.05, 1.05])

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 1, 1)
plt.plot([0,4],[0,4])

ax.scatter(output_sca1['Bars_Speech'], output_sca1['Bars_Speech_pred_t'], c = 'k' , marker = 'x')
ax.scatter(output_sca2['Bars_Speech'], output_sca2['Bars_Speech_pred_t'], c = 'g' , marker = 'x')
ax.scatter(output_sca3['Bars_Speech'], output_sca3['Bars_Speech_pred_t'], c = 'r' , marker = 'x')
ax.scatter(output_c['Bars_Speech'], output_c['Bars_Speech_pred_t'], c = 'b' , marker = 'x')

plt.xlim([-0.2, 4])
plt.ylim([-0.05, 4])

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 1, 1)
plt.plot([0,30],[0,30])
ax.scatter(output_c['Bars'], output_c['Bars_Total_pred_t'], c = 'b' , marker = 'x')
ax.scatter(output_sca1['Bars'], output_sca1['Bars_Total_pred_t'], c = 'k' , marker = 'x')
ax.scatter(output_sca2['Bars'], output_sca2['Bars_Total_pred_t'], c = 'g' , marker = 'x')
ax.scatter(output_sca3['Bars'], output_sca3['Bars_Total_pred_t'], c = 'r' , marker = 'x')
plt.xlim([-0.2, 30])
plt.ylim([-0.05, 30])

In [ ]:
output[['subject_id','Bars', 'Bars_Speech', 'Samples_no', 'Diagnosis','Prob_AT_tf_grad', 'Bars_Speech_pred_t','Bars_Total_pred_t']].head(200)

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, mean_absolute_error

In [ ]:
output_ = output

In [ ]:
output_agg = output.groupby(['subject_id'], as_index = False).median()

In [ ]:
output_agg

In [ ]:
output_agg.loc[output_agg['Diagnosis'] > 0,'Label_true'] = 1
output_agg.loc[output_agg['Diagnosis'] == 0,'Label_true'] = 0
output_agg.loc[output_agg['Prob_AT_tf_grad'] > 0.6,'Label_pred'] = 1
output_agg.loc[output_agg['Prob_AT_tf_grad'] <= 0.6,'Label_pred'] = 0

In [ ]:
output_agg

In [ ]:
f1_score(output_agg['Label_true'].to_numpy(), output_agg['Label_pred'].to_numpy(), average='weighted')

In [ ]:
roc_auc_score(output_agg['Label_true'].to_numpy(),output_agg['Prob_AT_tf_grad'].to_numpy(),average = 'weighted')

In [ ]:
mean_absolute_error(output_agg['Bars_Speech'].to_numpy(), output_agg['Bars_Speech_pred_t'].to_numpy())

In [ ]:
mean_absolute_error(output_agg['Bars'].to_numpy(), output_agg['Bars_Total_pred_t'].to_numpy())

In [ ]:
len(output_agg['subject_id'].unique())

In [ ]:
import glob

In [ ]:
#survey_lex = pd.read_csv(data_home_dir + 'Biogen_SCA_Speech_Data/' + 'surveylex_questionIDs.csv',encoding='latin1')
survey_lex = pd.read_excel(data_home_dir_at + 'Remote_AT_Speech_Data/' + 'surveylex_questionIDs.xlsx',engine='openpyxl')
taskslist = survey_lex['question'].unique()
#output_at = pd.DataFrame(columns = ['Date','subject_id','Neuropheno_ID','Bars','Bars_Speech','Sex','Diagnosis','DoB','Samples_no','Prob_AT_tf_grad', 'Bars_Speech_pred_t','Bars_Total_pred_t'])
output_at = pd.DataFrame(columns = ['Date','subject_id','Neuropheno_ID','Diagnosis','Samples_no','Prob_AT_tf_grad', 'Bars_Speech_pred_t','Bars_Total_pred_t'])
                                            
for tt in  taskslist: 
    if tt not in ["la_la_la","go_go_go","me_me_me", "lalala", "gogogo", "mememe","papapa"]:
        continue
    print(tt)
    pd.set_option('display.max_columns', 4)
    np.dtype(np.int16)
    task = str(tt)
    task_dir = str(tt)
    
    study_name = 'Remote_AT_Speech_Data'
    study_dir = data_home_dir_at + 'Remote_AT_Speech_Data/'
    survey_lex = pd.read_excel(study_dir + 'surveylex_questionIDs.xlsx',engine='openpyxl')

    survey_response_A_7 = pd.read_excel(study_dir + 'surveyA_7_responses_summary.xlsx',engine='openpyxl')
    survey_response_A_8_13 = pd.read_excel(study_dir + 'surveyA_8_13_responses_summary.xlsx',engine='openpyxl')
    survey_response_A_14 = pd.read_excel(study_dir + 'surveyA_14_responses_summary.xlsx',engine='openpyxl')
    survey_response_B_7 = pd.read_excel(study_dir + 'surveyB_7_responses_summary.xlsx',engine='openpyxl')
    survey_response_B_8_13 = pd.read_excel(study_dir + 'surveyB_8_13_responses_summary.xlsx',engine='openpyxl')
    survey_response_B_14 = pd.read_excel(study_dir + 'surveyB_14_responses_summary.xlsx',engine='openpyxl')
    survey_response_C_7 = pd.read_excel(study_dir + 'surveyC_7_responses_summary.xlsx',engine='openpyxl')
    survey_response_C_8_13 = pd.read_excel(study_dir + 'surveyC_8_13_responses_summary.xlsx',engine='openpyxl')
    #survey_response_C_14 = pd.read_excel(study_dir + 'surveyC_14_responses_summary.xlsx',engine='openpyxl')
    survey_response = pd.concat([survey_response_A_7, survey_response_A_8_13, survey_response_A_14, survey_response_B_7, survey_response_B_8_13, survey_response_B_14, survey_response_C_7, survey_response_C_8_13], ignore_index=True)

    survey_submissions_A_7 = pd.read_excel(study_dir + 'surveyA_7_submissions_summary.xlsx',engine='openpyxl')
    survey_submissions_A_8_13 = pd.read_excel(study_dir + 'surveyA_8_13_submissions_summary.xlsx',engine='openpyxl')
    survey_submissions_A_14 = pd.read_excel(study_dir + 'surveyA_14_submissions_summary.xlsx',engine='openpyxl')
    survey_submissions_B_7 = pd.read_excel(study_dir + 'surveyB_7_submissions_summary.xlsx',engine='openpyxl')
    survey_submissions_B_8_13 = pd.read_excel(study_dir + 'surveyB_8_13_submissions_summary.xlsx',engine='openpyxl')
    survey_submissions_B_14 = pd.read_excel(study_dir + 'surveyB_14_submissions_summary.xlsx',engine='openpyxl')
    survey_submissions_C_7 = pd.read_excel(study_dir + 'surveyC_7_submissions_summary.xlsx',engine='openpyxl')
    survey_submissions_C_8_13 = pd.read_excel(study_dir + 'surveyC_8_13_submissions_summary.xlsx',engine='openpyxl')
    #survey_submissions_C_14 = pd.read_excel(study_dir + 'surveyC_14_submissions_summary.xlsx',engine='openpyxl')
    survey_submissions = pd.concat([survey_submissions_A_7, survey_submissions_A_8_13, survey_submissions_A_14, survey_submissions_B_7, survey_submissions_B_8_13, survey_submissions_B_14, survey_submissions_C_7, survey_submissions_C_8_13], ignore_index=True)

    
#     Patient_info = pd.read_csv(study_dir + 'Subjects_Remote_Behavior.csv',encoding='latin1')
#     Patient_info_ = Patient_info.loc[:,['subject_id','neuropheno_id','Diagnosis_1']]
#     Patient_info_['Diagnosis_1'] = Patient_info_['Diagnosis_1'].apply(lambda x: 0 if x == 'Control' else 1)
#     Patient_info_ = Patient_info_.rename(columns={"Diagnosis_1": "diagnosis"})

    Patient_info = pd.read_csv('/home/kvattis/Desktop/Subjects_Remote_Behavior_AT.csv',encoding='latin1')
    Patient_info_ = Patient_info.loc[:,['subject_id','Neuropheno Study Link','diagnosis']] 
    Patient_info_['diagnosis'] = Patient_info_['diagnosis'].apply(lambda x: 0 if x == 'Control' else 1)
    Patient_info_ = Patient_info_.rename(columns={"diagnosis": "diagnosis", "Neuropheno Study Link" : 'neuropheno_id'})
    
    survey_lex_ = survey_lex.loc[:,['questionId','question']]
    survey_response_ = survey_response.loc[:,['sampleId','questionId', 'sessionId','createdDate']]
    survey_response_['createdDate'] = survey_response_['createdDate'].apply(lambda x: x[0:4] + '_' + x[5:7] + '_' + x[8:10] )
    survey_submissions_ = survey_submissions.loc[:,['sessionId','Your participant ID:']]
    survey_submissions_ = survey_submissions_.rename(columns={"Your participant ID:": "subject_id"})
    df = pd.merge(pd.merge(pd.merge(survey_response_,survey_lex_,on='questionId'), survey_submissions_, on = 'sessionId'),Patient_info_, on = 'subject_id')
    df.dropna(subset = ["sampleId"], inplace=True)
    df_task = df.loc[df['question'] == task][['sampleId','subject_id','neuropheno_id','diagnosis','createdDate']]
    df_task_numpy = df_task.to_numpy()
        
    list_diagnostic = ['Control', 'Ataxia', 'MSA', 'Other']

    
    #Going over all the files for the task
    list_names = []
    for entry in df_task_numpy:
        #print(entry)
        file_name = entry[0]
        ID = str(entry[1])

        if  not (pd.isnull(entry[2]) or entry[2] == 'NAN'):
            Neur_phen_id = str(int(entry[2]))
        else:
            Neur_phen_id = ID
        
        if int(Neur_phen_id) == 20094:
            continue
            
        if int(Neur_phen_id) in IDs_used:
            continue
        
        diagn = entry[3]
        date = entry[4]
        Date = date[:4] + date[5:7] + date[8:]
#         bars_total = (entry[5] + entry[6])/2.
#         bars_speech = (entry[7] + entry[8])/2./10.
#         sex = entry[9]
        #age = entry[10]
        
        path_to_file = study_dir + ID + '_' + date + '*_A/' + file_name + '.wav'
        path_to_file_flag = 0        
        
        #if os.path.exists(path_to_file):
        file_ = glob.glob(path_to_file)
        if file_: 
            path_to_file_flag = 1
            path_to_file = file_[0]
        
        if path_to_file_flag == 0:
            path_to_file = study_dir + ID + '_' + date + '*_B/' + file_name + '.wav'
            file_ = glob.glob(path_to_file)
            if file_: 
                path_to_file_flag = 1
                path_to_file = file_[0]
        
        if path_to_file_flag == 0:
            path_to_file = study_dir + ID + '_' + date + '*_C*/' + file_name + '.wav'
            file_ = glob.glob(path_to_file)
            if file_: 
                path_to_file_flag = 1
                path_to_file = file_[0]
            
        if path_to_file_flag == 0:
            path_to_file = study_dir + ID + '_' + date + '*_D/' + file_name + '.wav'
            file_ = glob.glob(path_to_file)
            if file_: 
                path_to_file_flag = 1
                path_to_file = file_[0]  
        
        if path_to_file_flag == 0: continue;
            
        #Assign label
        if diagn == 0:
            #diagn = 'Control'
            label = 0
        elif diagn == 1:
            #diagn = 'Ataxia'
            label = 1
        elif diagn == 2:
            #diagn = 'Ataxia'
            label = 1
        elif diagn == 3:
            #diagn = 'Ataxia'
            label = 1
        elif diagn == 7:
            #diagn = 'MSA'
            label = 7
        else:
            #diagn = 'Other'
            label = 5
        
        if label not in [0,1]:
            continue
        
        if label == 0:
            bars_total = 0
            bars_speech = 0 

        if Neur_phen_id == ID:
            no_samples = speech_analyzer.load(path_to_file,ID)
            Neur_phen_id = None
        else:
            no_samples = speech_analyzer.load(path_to_file,Neur_phen_id)
        
        if no_samples == 0: 
            continue
            
#         output_at = output_at.append({'Date' : Date, 'subject_id' : ID, 'Neuropheno_ID' : Neur_phen_id, 'Bars': bars_total, 'Bars_Speech':bars_speech, 'Sex': sex, 'Diagnosis': diagn,'Samples_no' : speech_analyzer.sample_size_s, 'Prob_AT_tf_grad': speech_analyzer.classify(),
#                                'Bars_Speech_pred_t': speech_analyzer.BARS_speech(), 'Bars_Total_pred_t': speech_analyzer.BARS_total()}, 
#                 ignore_index = True)

        output_at = output_at.append({'Date' : Date, 'subject_id' : ID, 'Neuropheno_ID' : Neur_phen_id, 'Diagnosis': diagn,'Samples_no' : speech_analyzer.sample_size_s, 'Prob_AT_tf_grad': speech_analyzer.classify(),
                               'Bars_Speech_pred_t': speech_analyzer.BARS_speech(), 'Bars_Total_pred_t': speech_analyzer.BARS_total()}, 
                ignore_index = True)
        
output_at = output_at.astype(float).groupby(['Date','subject_id'], as_index = False).median().sort_values(by=['subject_id'])

In [ ]:
output_at.columns

In [ ]:
output_at

In [ ]:
output_at[['subject_id', 'Samples_no', 'Diagnosis','Prob_AT_tf_grad', 'Bars_Speech_pred_t','Bars_Total_pred_t']].head(200)

In [ ]:
output_at[['Diagnosis','Prob_AT_tf_grad']]

In [ ]:
output_at_agg = output_at.groupby(['subject_id'], as_index = False).median()

In [ ]:
output_at_agg['Label_true'].value_counts()

In [ ]:
output_agg['Label_true'].value_counts()

In [ ]:
output_at_agg.loc[output_at_agg['Diagnosis'] > 0,'Label_true'] = 1
output_at_agg.loc[output_at_agg['Diagnosis'] == 0,'Label_true'] = 0
output_at_agg.loc[output_at_agg['Prob_AT_tf_grad'] > 0.6,'Label_pred'] = 1
output_at_agg.loc[output_at_agg['Prob_AT_tf_grad'] <= 0.6,'Label_pred'] = 0

In [ ]:
f1_score(output_at_agg['Label_true'].to_numpy(), output_at_agg['Label_pred'].to_numpy(), average='weighted')

In [ ]:
roc_auc_score(output_at_agg['Label_true'].to_numpy(),output_at_agg['Prob_AT_tf_grad'].to_numpy(),average = 'weighted')

In [ ]:
len(output_at[['Label_true','Label_pred']])

In [ ]:
len(output[['Label_true','Label_pred']])

In [ ]:
output_agg.columns

In [ ]:
output_at_agg.columns

In [ ]:
output_at_agg.loc[output_at_agg['Diagnosis'] > 0,'Bars'] = np.nan
output_at_agg.loc[output_at_agg['Diagnosis'] == 0,'Bars'] = 0
output_at_agg.loc[output_at_agg['Diagnosis'] > 0,'Bars_Speech'] = np.nan
output_at_agg.loc[output_at_agg['Diagnosis'] == 0,'Bars_Speech'] = 0

In [ ]:
 output_merged = pd.concat([output_agg[['Label_true','Label_pred','Prob_AT_tf_grad','Bars','Bars_Speech','Bars_Speech_pred_t', 'Bars_Total_pred_t']],output_at_agg[['Label_true','Label_pred','Prob_AT_tf_grad','Bars','Bars_Speech','Bars_Speech_pred_t', 'Bars_Total_pred_t']]],ignore_index=False)

In [ ]:
f1_score(output_merged['Label_true'].to_numpy(), output_merged['Label_pred'].to_numpy(), average='weighted')

In [ ]:
roc_auc_score(output_merged['Label_true'].to_numpy(),output_merged['Prob_AT_tf_grad'].to_numpy(),average = 'weighted')

In [ ]:
output_merged_filt = output_merged[~output_merged['Bars_Speech'].isna()]

In [ ]:
mean_absolute_error(output_merged_filt['Bars_Speech'].to_numpy(), output_merged_filt['Bars_Speech_pred_t'].to_numpy())

In [ ]:
mean_absolute_error(output_merged_filt['Bars'].to_numpy(), output_merged_filt['Bars_Total_pred_t'].to_numpy())

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(output_merged_filt['Bars_Speech'].to_numpy(), output_merged_filt['Bars_Speech_pred_t'].to_numpy())

In [ ]:
r2_score(output_merged_filt['Bars'].to_numpy(), output_merged_filt['Bars_Total_pred_t'].to_numpy())

In [ ]:
output_merged_filt[output_merged_filt['Label_true'] == 1]['Bars_Speech'].max()

In [ ]:
output_merged['Label_true'].value_counts()

In [ ]:
output_merged[['Label_true','Label_pred']]

In [ ]:
taskslist

In [ ]:
len(output_at['subject_id'].unique())

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 1, 1)
ax.scatter(output_merged[output_merged['Label_true'] == 0]['Bars_Speech'], output_merged[output_merged['Label_true'] == 0]['Prob_AT_tf_grad'], c = 'b' , marker = 'x')
ax.scatter(output_merged[output_merged['Label_true'] == 1]['Bars_Speech'], output_merged[output_merged['Label_true'] == 1]['Prob_AT_tf_grad'], c = 'r' , marker = 'x')
plt.ylim([-0.05, 1.05])

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 1, 1)
ax.scatter(output_merged_filt[output_merged_filt['Label_true'] == 0]['Bars_Speech'], output_merged_filt[output_merged_filt['Label_true'] == 0]['Bars_Speech_pred_t'], c = 'b' , marker = 'x')
ax.scatter(output_merged_filt[output_merged_filt['Label_true'] == 1]['Bars_Speech'], output_merged_filt[output_merged_filt['Label_true'] == 1]['Bars_Speech_pred_t'], c = 'r' , marker = 'x')
plt.plot([0,4],[0,4])
#plt.ylim([-0.05, 1.05])

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 1, 1)
ax.scatter(output_merged_filt[output_merged_filt['Label_true'] == 0]['Bars'], output_merged_filt[output_merged_filt['Label_true'] == 0]['Bars_Total_pred_t'], c = 'b' , marker = 'x')
ax.scatter(output_merged_filt[output_merged_filt['Label_true'] == 1]['Bars'], output_merged_filt[output_merged_filt['Label_true'] == 1]['Bars_Total_pred_t'], c = 'r' , marker = 'x')
plt.plot([0,30],[0,30])
#plt.ylim([-0.05, 1.05])